# AF_unmasked: a simplified notebook

<img src="https://github.com/clami66/AF_unmasked/raw/main/fig/header.png" height="200">

This notebook allows to run AF_unmasked on multimeric sequences and templates of your choice. Not all features implemented on the command line version of AF_unmasked are currently available on the notebook, but more will come later.

This version of AF_unmasked relies on MMseqs2 alignments, run by the [ColabFold](https://github.com/sokrypton/ColabFold) MSA server. Some of the code on this notebook is also based on ColabFold.

If you use this version of AF_unmasked in your research, please cite the articles under the "References" section below.


In [ ]:
#@title Step 1: Install dependencies
import os
from sys import version_info, path
python_version = f"{version_info.major}.{version_info.minor}"
PYTHON_VERSION = python_version

os.system("git clone -b notebook https://github.com/clami66/AF_unmasked.git")

if not os.path.isfile("COLABFOLD_READY"):
  print("installing colabfold...")
  os.system("pip install -q --no-warn-conflicts biopython==1.79 'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'")
  if os.environ.get('TPU_NAME', False) != False:
    os.system("pip uninstall -y jax jaxlib")
    os.system("pip install --no-warn-conflicts --upgrade dm-haiku==0.0.10 'jax[cuda12_pip]'==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold")
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold")
  os.system("touch COLABFOLD_READY")


if not os.path.isfile("CONDA_READY"):
  print("installing conda...")
  os.system("wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh")
  os.system("bash Mambaforge-Linux-x86_64.sh -bfp /usr/local")
  os.system("mamba config --set auto_update_conda false")
  os.system("touch CONDA_READY")

print("installing conda packages...")
!mamba install -y -c conda-forge -c bioconda hmmer kalign2=2.04 hhsuite=3.3.0 &> /dev/null

path.insert(0,'/content/AF_unmasked')

import pickle
import shutil
import importlib_metadata
from pathlib import Path

import ipywidgets as widgets
from google.colab import files

from AF_unmasked.alphafold.data.prepare_templates import *
from AF_unmasked.alphafold.data.mmseqs_2_uniprot import *
from Bio import Align, SeqIO, AlignIO
from Bio.PDB.mmcifio import MMCIFIO

from colabfold.batch import get_msa_and_templates, msa_to_str
from colabfold.utils import DEFAULT_API_SERVER, get_commit
from colabfold.download import download_alphafold_params

from AF_unmasked.run_alphafold import predict_structure
from AF_unmasked.alphafold.data import pipeline, pipeline_multimer

from AF_unmasked.alphafold.data.tools import hmmsearch, jackhmmer
from AF_unmasked.alphafold.data import templates

from AF_unmasked.alphafold.model import model, data, config



In [2]:
#@title Step 2: Input protein sequences
jobname = "H1142" #@param {type:"string"}

#@markdown - Only fasta inputs are accepted (see example below)
#@markdown - Leave the fasta input as is to run example from AF_unmasked paper (H1142)
fasta_input = widgets.Textarea(
    value=""">H1142_A
GLEKDFLPLYFGWFLTKKSSETLRKAGQVFLEELGNHKAFKKELRHFISGDEPKEKLELVSYFGKRPPGVLHCTTKFCDYKAAGAEEYAQQEVVKRSYGKAFKLSISALFVTPKTAGAQVVLTDQELQLWPSDLDKPSASEGLPPGSRAHVTLGCAADVQPVQTGLDLLDILQQVKGGSQGEAVGELPRGKLYSLGKGRWMLSLTKKMEVKAIFTGYYG
>H1142_B
EVQLEESGGGLVQAGGSLTLSCAASGFTFDDYAMGWYRQAPGKERVGVSCISRTDGYTYYLDSVKGRFTISTDHAKHTVYLQMNNLKPDDTGLYYCAADADPEYGSRCPDPYYGMDYWGKGILVTVSS""",
    placeholder='Input protein sequences in fasta format',
    description='Fasta input:',
    disabled=False,
    layout=widgets.Layout(height="200px", width="auto")
)

display(fasta_input)




Textarea(value='>H1142_A\nGLEKDFLPLYFGWFLTKKSSETLRKAGQVFLEELGNHKAFKKELRHFISGDEPKEKLELVSYFGKRPPGVLHCTTKFCDYKAAG…

In [ ]:
#@title Step 3: Upload template structure (.pdb or .cif)
example_pdb = False #@param {type:"boolean"}
#@markdown - Check box to use example from AF_unmasked paper (H1142.pdb)
#@markdown - Otherwise, run this cell and click on the "Browse" button below to upload a template pdb

out_dir = Path(f"{jobname}")
out_dir.mkdir(parents=True, exist_ok=True)
sequences = fasta_input.value

targets = out_dir.joinpath(f"{jobname}.fasta")

with open(targets, "w") as out:
    out.write(sequences)

if not example_pdb:
  uploaded = files.upload()
  pdb = Path(list(uploaded.keys())[0])
else:
  pdb = Path("/content/AF_unmasked/examples/H1142/H1142.pdb")

if pdb.name.endswith(".pdb"):
  template_format = "pdb"
elif pdb.name.endswith(".cif"):
  template_format = "cif"
else:
  raise ValueError("Template must be in .pdb or .cif format")

template = out_dir.joinpath(pdb.name)
_ = shutil.copyfile(pdb, template)

In [4]:
#@title Step 4: Set other parameters
model_type = "alphafold2_multimer_v2" #@param ["alphafold2_multimer_v2", "alphafold2_multimer_v3"]

predictions_per_model = 1 #@param {type:"integer"}
num_recycles = 3 #@param {type:"integer"}
recycle_early_stop_tolerance = 0.5 #@param {type:"number"}
use_dropout = True #@param {type:"boolean"}

#@markdown #### MSA settings
msa_mode = "mmseqs2_uniref" #@param ["no_MSA", "mmseqs2_uniref", "mmseqs2_uniref_env"]

#@markdown -  decrease `msa_depth` to rely more on template information (min: 1)
msa_depth = 10 #@param {type:"integer"}
#@markdown - Pairing of MSAs is currently not allowed to enforce template contraints

if msa_depth == "auto": msa_depth = None

#@markdown Template options
inpaint_clashes = True #@param {type:"boolean"}
align = True #@param {type:"boolean"}
superimpose = False #@param {type:"boolean"}
align_tool = "blast"

#@markdown #### Save settings
save_to_google_drive = False #@param {type:"boolean"}

if save_to_google_drive:
  from pydrive2.drive import GoogleDrive
  from pydrive2.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("You are logged into Google Drive and are good to go!")


In [ ]:
#@title Step 5: provide mapping between target sequences and template chains

# template data
template_model = load_PDB(template, is_mmcif=(template_format == "cif"))
template_chains = [c.id for c in template_model]
remove_extra_chains(template_model, template_chains)
remove_hetatms(template_model)
template_sequences = [
        get_fastaseq(template_model, chain) for chain in template_chains
    ]

# target data
target_chains, target_sequences, target_models = get_target_data(
            [str(targets)],
            chains=None,
            is_fasta=True,
            is_mmcif=False,
        )
template_preview = [f"Chain {template_chain} (seq: {template_seq[:10]}...)" for template_chain, template_seq in zip(template_chains, template_sequences)]
template_c = [widgets.Dropdown(options=template_preview, value=template_preview[i]) for i, ch in enumerate(target_chains)]

for i, widget in enumerate(template_c):
  print(f"Select template chain for fasta sequence {i+1} (seq: {target_sequences[i][:10]}...)")
  display(widget)

In [ ]:
#@title Step 6: Perform sequence-template alignments and check the results
#@markdown - Make sure to double check the alignments in the output to see that they are correct

template_chains = [temp.value.split()[1] for temp in template_c]

if len(template_chains) != len(set(template_chains)):
  raise ValueError("Must select a different template chain for each fasta sequence")

append = False
mmcif_path = Path(out_dir, "template_data", "mmcif_files")
mmcif_path.mkdir(parents=True, exist_ok=True)
next_id = get_next_id(mmcif_path) if append else "0000"

io = MMCIFIO()
template_mmcif_path = os.path.join(
    out_dir, "template_data", "mmcif_files", f"{next_id}.cif"
)

if inpaint_clashes:
    template_model = detect_and_remove_clashes(template_model)
    template_sequences = [
        get_fastaseq(template_model, chain) for chain in template_chains
    ]

io.set_structure(template_model)
io.save(template_mmcif_path)

fix_mmcif(
    template_mmcif_path, template_chains, template_sequences, "2100-01-01"
)

pdb_seqres_path = Path(out_dir, "template_data", "pdb_seqres.txt").resolve()
write_seqres(
    pdb_seqres_path,
    template_sequences,
    template_chains,
    seq_id=next_id,
    append=append,
)

# extra flagfile for AF usage
af_flagfile_path = Path(out_dir, "template_data", "templates.flag")
if not af_flagfile_path.is_file():  # don't overwrite file if already there
    with open(af_flagfile_path, "w") as flagfile:
        flagfile.write(f"--template_mmcif_dir={mmcif_path.resolve()}\n")
        flagfile.write(f"--pdb_seqres_database_path={pdb_seqres_path}\n")
        if align:  # means we are not going to let AF overwrite pdb_hits.sto
            flagfile.write("--use_precomputed_msas\n")

if align:

    assert len(target_chains) == len(
        template_chains
    ), f"The number of chains to align from target ({target_chains}) doesn't match the number of chains in the template ({template_chains}). Make sure that the files contain the same number of chains or select the chains that should be paired with --target_chains, --template_chains"
    for (
        i,
        (
            template_chain,
            template_sequence,
            target_chain,
            target_sequence,
            target_model,
        ),
    ) in enumerate(
        zip(
            template_chains,
            template_sequences,
            target_chains,
            target_sequences,
            target_models,
        )
    ):
        msa_chain = ascii_upperlower[i]
        this_template_model = pickle.loads(pickle.dumps(template_model, -1))
        this_target_model = pickle.loads(pickle.dumps(target_model, -1))
        print(f"Aligning fasta sequence {i+1} (seq: {target_sequence[0:10]}...) to template chain {template_chain} (seq: {template_sequence[0:10]}...)")
        alignment = do_align(
            template_sequence,
            this_template_model,
            target_sequence,
            this_target_model,
            alignment_type="blast",
        )
        sto_alignment = format_alignment_stockholm(
            alignment, hit_id=next_id, hit_chain=template_chain
        )


        msa_path = f"msas/{msa_chain}"

        # write alignment to file
        Path(out_dir, msa_path).mkdir(parents=True, exist_ok=True)
        with open(
            Path(out_dir, msa_path, "pdb_hits.sto"),
            mode="a" if append else "w",
        ) as pdb_hits:
            for line in sto_alignment:
                pdb_hits.write(line)

In [ ]:
#@title Step 7: Run the prediction with AF_unmasked

if msa_mode == "no_MSA": # same as "no_MSA" on the AF_unmasked paper
  unpaired_msa = []
  for i, ts in enumerate(target_sequences):
    unpaired_msa.append(f"> seq_{i}\n{ts}")
else: # Alignments rely on ColabFold's MSA server
  print("Querying ColabFold's MSA server")
  msa_lines = None
  use_templates = False
  custom_template_path = None
  pair_mode = "unpaired"
  pairing_strategy = "greedy"
  host_url = DEFAULT_API_SERVER
  version = importlib_metadata.version("colabfold")
  commit = get_commit()
  if commit:
      version += f" ({commit})"
  user_agent = f"colabfold/{version}"

  unpaired_msa, paired_msa, query_seqs_unique, query_seqs_cardinality, template_features = get_msa_and_templates(jobname, target_sequences, msa_lines, out_dir, msa_mode, use_templates,
                          custom_template_path, pair_mode, pairing_strategy, host_url, user_agent)

for chain, sequence, msa in zip(target_chains, target_sequences, unpaired_msa):
  out_dir.joinpath(f"msas/{chain}/bfd_uniref_hits.a3m").write_text(msa)
  pseudo_uniprot = open(out_dir.joinpath(f"msas/{chain}/uniprot_hits.a3m"), "w")
  pseudo_uniprot.write(f"> {chain}\n")
  pseudo_uniprot.write(str(sequence))
  pseudo_uniprot.close()

  input_handle  = open(out_dir.joinpath(f"msas/{chain}/uniprot_hits.a3m"), "r")
  output_handle = open(out_dir.joinpath(f"msas/{chain}/uniprot_hits.sto"), "w")

  alignments = AlignIO.parse(input_handle, "fasta")
  AlignIO.write(alignments, output_handle, "stockholm")

  output_handle.close()
  input_handle.close()

data_dir = Path("/content")
if not glob.glob(f"{data_dir}/params/*_finished.txt"):
  print("downloading AF parameters...")
  download_alphafold_params(model_type, data_dir)

template_searcher = hmmsearch.Hmmsearch(
    binary_path=shutil.which("hmmsearch"),
    hmmbuild_binary_path=shutil.which("hmmbuild"),
    database_path=out_dir.joinpath(f"template_data/pdb_seqres.txt"))

template_featurizer = templates.HmmsearchHitFeaturizer(
    mmcif_dir=mmcif_path.resolve(),
    max_template_date="2100-01-01",
    max_hits=4,
    kalign_binary_path=shutil.which("kalign"),
    release_dates_path=None,
    obsolete_pdbs_path=None)

monomer_data_pipeline = pipeline.DataPipeline(
      jackhmmer_binary_path=shutil.which("jackhmmer"),
      hhblits_binary_path=shutil.which("hhblits"),
      uniref90_database_path=".",
      mgnify_database_path="",
      bfd_database_path="",
      uniref30_database_path="",
      small_bfd_database_path="",
      template_searcher=template_searcher,
      template_featurizer=template_featurizer,
      use_small_bfd=False,
      use_precomputed_msas=True,
      mgnify_max_hits=1,
      uniref_max_hits=1,
      bfd_max_hits=msa_depth,
      no_uniref=True,
      no_mgnify=True)

data_pipeline = pipeline_multimer.DataPipeline(
        monomer_data_pipeline=monomer_data_pipeline,
        jackhmmer_binary_path=shutil.which("jackhmmer"),
        uniprot_database_path=None,
        use_precomputed_msas=True,
        max_uniprot_hits=1,
        separate_homomer_msas=True)

model_names = ["model_1_multimer_v2", "model_5_multimer_v2"] if model_type == "alphafold2_multimer_v2" else ["model_1_multimer_v3", "model_5_multimer_v3"]

model_runners = {}

for model_name in model_names:
    model_config = config.model_config(model_name)
    model_config.model.num_ensemble_eval = 1
    model_config.model.embeddings_and_evoformer.cross_chain_templates = True
    model_config.model.num_recycle = num_recycles
    model_config.model.global_config.eval_dropout = use_dropout
    model_config.model.recycle_early_stop_tolerance = recycle_early_stop_tolerance

    model_params = data.get_model_haiku_params(
        model_name=model_name, data_dir=str(data_dir))
    model_runner = model.RunModel(model_config, model_params)
    for i in range(predictions_per_model):
      model_runners[f'{model_name}_pred_{i}'] = model_runner

predict_structure(
        fasta_path=targets,
        fasta_name=jobname,
        output_dir_base="/content",
        data_pipeline=data_pipeline,
        model_runners=model_runners,
        benchmark=False,
        random_seed=0,
        models_to_relax=None)

In [12]:
#@title Step 8: Download results
exclude_pickles = True #@param {type:"boolean"}
extra_zip_flags = "-x '*.pkl'" if exclude_pickles else ""
results_zip = f"{jobname}_AF_unmasked.zip"
os.system(f"zip {extra_zip_flags} -r {results_zip} {jobname}")

files.download(f"{jobname}_AF_unmasked.zip")
if save_to_google_drive == True and drive:
  uploaded = drive.CreateFile({'title': f"{jobname}_AF_unmasked.zip"})
  uploaded.SetContentFile(f"{jobname}_AF_unmasked.zip")
  uploaded.Upload()
  print(f"Uploaded {jobname}_AF_unmasked.zip to Google Drive with ID {uploaded.get('id')}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title References

"""
@article{af_unmasked,
  title={Unmasking AlphaFold: integration of experiments and predictions in multimeric complexes},
  author={Mirabello, Claudio and Wallner, Bj{\"o}rn and Nystedt, Bj{\"o}rn and Azinas, Stavros and Carroni, Marta},
  journal={bioRxiv},
  pages={2023--09},
  year={2023},
  publisher={Cold Spring Harbor Laboratory}
}

@article{af2,
  title={Highly accurate protein structure prediction with {AlphaFold}},
  author={Jumper, John and Evans, Richard and Pritzel, Alexander and Green, Tim and Figurnov, Michael and Ronneberger, Olaf and Tunyasuvunakool, Kathryn and Bates, Russ and {\v{Z}}{\'\i}dek, Augustin and Potapenko, Anna and others},
  journal={Nature},
  volume={596},
  number={7873},
  pages={583--589},
  year={2021},
  publisher={Nature Publishing Group UK London}
}

@article{af2_multimer,
  title={Protein complex prediction with {AlphaFold-Multimer}},
  author={Evans, Richard and O’Neill, Michael and Pritzel, Alexander and Antropova, Natasha and Senior, Andrew and Green, Tim and {\v{Z}}{\'\i}dek, Augustin and Bates, Russ and Blackwell, Sam and Yim, Jason and others},
  journal={BioRxiv},
  pages={2021--10},
  year={2021},
  publisher={Cold Spring Harbor Laboratory}
}

@article{colabfold,
  title={ColabFold: making protein folding accessible to all},
  author={Mirdita, Milot and Sch{\"u}tze, Konstantin and Moriwaki, Yoshitaka and Heo, Lim and Ovchinnikov, Sergey and Steinegger, Martin},
  journal={Nature methods},
  volume={19},
  number={6},
  pages={679--682},
  year={2022},
  publisher={Nature Publishing Group US New York}
}
"""